# Семинар_9 Оптимизация памяти. Ускорение вычислений.

Цель семинара: освоить техники оптимизация памяти и ускорения вычислений.

План семинара:

* Практика - ускорение и оптимизация загрузки/сохранения датасетов
* Практика - numpy where
* Практика - numpy vectorize
* Практика - numpy select
* Практика - оптимизация pd.groupby()

* Подведение итогов - проанализируем и обсудим результаты

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import time

pd.set_option('display.max_columns', None)

# 1. Ускорение загрузки/сохранения датасетов. (20 мин)

Для начала подгрузим датасет.
Мы будем использовать датасет [транзакции](https://www.kaggle.com/datasets/ranunculusrepens/transactions-data). Это не очень большой датасет. Но больше, чем тот, с которым мы работали ранее.
* Вам нужно найти и подгрузить его в ноутбук.

Представим, что наш датасет не влез в оперативую память. Мы хотим посмотреть только часть этого датасета. 
* Загрузите датафрейм, прочитав с диска только 10000 строк.

[дока](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html)

In [ ]:
transactions = pd.read_csv('/kaggle/input/transactions-data/transactions.csv',
                  ####)

Если бы датасет был огромный, мы могли бы работать с небольшими частями

* Какие столбцы можно оптимизировать?

Подгрузим весь датасет

* Внимательно изучите данные. Попытайтесь понять, что это за данные и какой тип данных в каждой из колонок. 

In [ ]:
transactions = pd.read_csv('/kaggle/input/transactions-data/transactions.csv')
transactions

* Посмотрим сколько датасет занимает места.

In [ ]:
### ваш код

* Проверьте, можно ли сконвертировать колонку transaction_amt во float32 или float16
* Найдите макимальное и минимальное значение в колонке и убедитесь, что они находятся в пределах максимального и минимального значения для float32 или float16. можно использовать [np.finfo](https://numpy.org/doc/stable/reference/generated/numpy.finfo.html) для получения макс и мин значенийй для типа
* Приведите к оптимальному типу
* Измерьте на сколько процентов улучшилось использование памяти

In [ ]:
### ваш код

* Переведем transaction_dttm в datetime

In [ ]:
start_mem = transactions.memory_usage().sum() / 1024**2
print(start_mem)

transactions['transaction_dttm'] = ### ваш код

end_mem = transactions.memory_usage().sum() / 1024**2
print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))

Переведем столбцы user_id, mcc_code и currency_rk в категорикал. (mcc_code и currency_rk по сути категориальные столбцы)

In [ ]:
cols2conv = ['mcc_code', 'currency_rk', 'user_id']
for col in cols2conv:
    print("converting", col.ljust(30), "size: ", round(transactions[col].memory_usage(deep=True)*1e-6,2), end="\t")
    transactions[col] = ### ваш код
    print("->\t", round(transactions[col].memory_usage(deep=True)*1e-6,2))

Таким образом, мы значительно уменьшили кол-во занимаемой памяти в нашем датасете. 
При следующих чтених мы можем автоматом ковертировать столбцы в нужные намм форматы.
* вызовите pd read_csv, при этом укажите в каких типах столбцы вы хотите получить
* так же для экономии места можно столбец `user_id` использовать как индекс - укажите это в методе read_csv.
* не забываем про дату

[дока](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html)

In [ ]:
transactions = pd.read_csv('/kaggle/input/transactions-data/transactions.csv',
                               ### ваш код)
transactions.info()

Сколько места теперь занимает датасет?

Можно пользоваться такой функцией для уменьшения размера датасета. 

In [ ]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))

    for col in df.columns:
        col_type = df[col].dtype.name

        if col_type not in ['object', 'category', 'datetime64[ns, UTC]']:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))

    return df

# 2. Метод numpy where() (10 мин)

В некоторых случая скорости работы пандас не хватает. 
В таких случаях можно ускорять используя numpy, [polars](https://pola.rs/), ускорять вычисления на видеокартах с помощью [rapids](https://rapids.ai/).

Далее попрактикуемся в ускорении расчетов с помощью numpy.

* Напишите функцию, которая возвращает user_id, если данная строка датафрейма удовлетворяет условию, иначе -1. И примените ее на наш датасет.
* Сначала решите задачу методами Pandas и замерьте скорость. Потом решите используя [np.where](https://numpy.org/doc/stable/reference/generated/numpy.where.html). И сравните скорости.
* Условие: currency_rk ==48, mcc_code == 5411, transaction_amt > 0
* Мы будем применять эту функцию методом apply к нашему датасету.

Обрежем датасет для скорости выполнения

In [ ]:
transactions = transactions = pd.read_csv('/kaggle/input/transactions-data/transactions.csv',
                  nrows=100000, parse_dates=['transaction_dttm'])

Примените функцию `reduce_mem_usage` к датасету

In [ ]:
transactions = ???

In [ ]:
def get_where(transactions):
    ### ваш код. используем Pandas
    mask = ()
    transactions.loc[~mask, 'result_column'] = -1
    transactions.loc[mask, 'result_column'] = transactions['user_id']

    return transactions['result_column']

In [ ]:
%%time

result = transactions.apply(get_where, axis=1)

In [ ]:
def get_where(transactions):
    ### ваш код. используем numpy
    mask = ()
    return np.where()

In [ ]:
%%time
result = transactions.apply(get_where, axis=1)

Какие результаты? Как думаете почему так получилось?

# 3. Метод numpy vectorize() (10 мин)

Теперь попробуем [np.vectorize()](https://numpy.org/doc/stable/reference/generated/numpy.vectorize.html)

Векторизация функции в NumPy позволяет применять функцию к каждому элементу массива или набора массивов.Векторизация ускоряет выполнение по сравнению с явным использованием циклов в Python, потому что NumPy оптимизирован для вычислений на массивах.

У нас есть функция (в след окне для кода). Перепишите ее в векторизованной форме и измерьте изменение в скорости.

In [ ]:
%%time
def f(x):
    if '49' == x['currency_rk']:
        if x['mcc_code'] == '5411':
            return 0
        else:
            if x['transaction_amt'] > 0:
                return 1
            return np.nan
    elif x['transaction_amt'] > 100:
        return 2
    else:
        return 3
    
preds = transactions.apply(f, axis=1)

In [ ]:
%%time

def f(currency_rk, mcc_code, transaction_amt):
    
    
vectfunc = np.vectorize(f)

preds = vectfunc(transactions['currency_rk'], transactions['mcc_code'], transactions['transaction_amt'])

# 4. Метод numpy select() (10 мин)

Если работа ведется с очень большими массивами и требуется максимальная производительность, np.select() скорее всего будет лучшим выбором по сравнению с np.vectorize().

In [ ]:
def f(x):
    if '49' == x['currency_rk']:
        if x['mcc_code'] == '5411':
            return 0
        else:
            if x['transaction_amt'] > 0:
                return 1
            return np.nan
    elif x['transaction_amt'] > 100:
        return 2
    else:
        return 3

Напишите conditions, choices, которые нужно будет использовать в np.select(), чтобы посчитать нашу функцию

[дока](https://numpy.org/doc/stable/reference/generated/numpy.select.html)

In [ ]:
%%time
conditions = ### ваш код

choices = ### ваш код

preds = np.select(conditions, choices, default=3)

# 5. Оптимизируем pd.groupby() (15 мин)

Задача: Написать функцию, которая для каждой группы (по `user_id`) будет считать для колонки `transaction_amt`: 
* среднее 
* сумму 
* кол-во элементов в группе. <br>
Сначала посчитайте используя pd.groupby, а затем попробуйте numpy векторизацию. И сравните время работы.

In [ ]:
%%time
def calculate_stats_groupby(df):
    ### ваш код
    return stats

stats_groupby = calculate_stats_groupby(transactions)

In [ ]:
stats_groupby[stats_groupby['count']>0]

Для удобства закодируем `user_id`

In [ ]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
transactions['user_id'] = label_encoder.fit_transform(transactions['user_id'])

Кол-во, сумму и среднее можно посчитать, например, при помощи [np.bincount](https://numpy.org/doc/stable/reference/generated/numpy.bincount.html).

In [ ]:
%%time
def calculate_stats_vectorized(df):
    cnt =
    sums =
    mean =

    res = pd.DataFrame({
        'user_id': np.arange(df['user_id'].max() + 1),
        'mean': mean,
        'sum': sums,
        'count': cnt,
    })
    return res

stats_vectorized = calculate_stats_vectorized(transactions)

In [ ]:
stats_vectorized

# 6. tqdm & pandarallel (5 мин)

* open source
* распараллеливает операции пандас на все доступные cpu
* [github](https://github.com/nalepae/pandarallel) проекта
* [дока](https://nalepae.github.io/pandarallel/)

In [ ]:
!pip install pandarallel -q

In [ ]:
# посчитаем насколько сумма транзакции меньше максимальной
def get_transaction_diff(x):
    time.sleep(0.0001)
    return 100000 - x

In [ ]:
%%time
transactions['diff'] = transactions['transaction_amt'].apply(get_transaction_diff)

Непонятно что происходит? Идет ли процесс, сколько ждать, может ячейка зависла?

### Progress apply

In [ ]:
%%time
from tqdm import tqdm
tqdm.pandas()


transactions['diff'] = transactions['transaction_amt'].progress_apply(get_transaction_diff)

## Parallel apply

Правильно импортируйте и инициализируйте Pandarallel
* Включите отображение прогресс-бара
* Укжите число доступных ядер CPU в вашем ноутбуке. 

In [ ]:
import 

pandarallel.initialize()

Вычислим признак с помощью Pandarallel. Как изменилось время?

In [ ]:
%%time
transactions['diff'] = ### ваш код